In [1]:
import os
import shutil
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

Splits images from the raw-img source directory into training (80%) and test (20%) samples, creating the corresponding train and test folders, and copies the images to the required directories if the splitting has not been done yet

In [3]:
base_dir = "./data/vision/raw-img"
dataset_dir = "./data/vision/dataset"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")


def is_class_split(train_class_path, test_class_path):
    return os.path.exists(train_class_path) and os.path.exists(test_class_path) and \
           len(os.listdir(train_class_path)) > 0 and len(os.listdir(test_class_path)) > 0


os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


split_ratio = 0.8  # 80% train, 20% test

if not os.path.exists(base_dir):
    print(f"The {base_dir} directory was not found. Check the path to the dataset.")
else:
    for class_name in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_name)
        if os.path.isdir(class_path):
            train_class_dir = os.path.join(train_dir, class_name)
            test_class_dir = os.path.join(test_dir, class_name)

            if is_class_split(train_class_dir, test_class_dir):
                print(f"Skip {class_name} (already split)")
                continue

            images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
            if not images:
                print(f"Missing {class_name} (no images in source folder)")
                continue

            random.shuffle(images)
            split_index = int(len(images) * split_ratio)
            train_images = images[:split_index]
            test_images = images[split_index:]

            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(test_class_dir, exist_ok=True)

            for img in train_images:
                src = os.path.join(class_path, img)
                dst = os.path.join(train_class_dir, img)
                shutil.copy(src, dst)

            for img in test_images:
                src = os.path.join(class_path, img)
                dst = os.path.join(test_class_dir, img)
                shutil.copy(src, dst)

            print(f"{class_name}: {len(train_images)} for train, {len(test_images)} for test")

    print(f"Dataset successfully split and saved to {dataset_dir}")

butterfly: 1689 for train, 423 for test
cat: 1334 for train, 334 for test
chicken: 2478 for train, 620 for test
cow: 1492 for train, 374 for test
dog: 3890 for train, 973 for test
elephant: 1156 for train, 290 for test
horse: 2098 for train, 525 for test
sheep: 1456 for train, 364 for test
spider: 3856 for train, 965 for test
squirrel: 1489 for train, 373 for test
Dataset successfully split and saved to ./data/vision/dataset


In [4]:
def count_images_in_folders(base_dir):
    class_counts = {}

    if not os.path.exists(base_dir):
        print(f"The {base_dir} directory was not found.")
        return class_counts

    for class_name in sorted(os.listdir(base_dir)):
        class_path = os.path.join(base_dir, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))

    return class_counts

train_counts = count_images_in_folders(train_dir)
test_counts = count_images_in_folders(test_dir)

if train_counts:
    print("The number of images in the train:")
    for class_name, count in train_counts.items():
        print(f"  {class_name}: {count}")
else:
    print("Train-directory is empty or missing.")

if test_counts:
    print("\n Number of images in test:")
    for class_name, count in test_counts.items():
        print(f"  {class_name}: {count}")
else:
    print("Test directory is empty or missing.")


The number of images in the train:
  butterfly: 1689
  cat: 1334
  chicken: 2478
  cow: 1492
  dog: 3890
  elephant: 1156
  horse: 2098
  sheep: 1456
  spider: 3856
  squirrel: 1489

 Number of images in test:
  butterfly: 423
  cat: 334
  chicken: 620
  cow: 374
  dog: 973
  elephant: 290
  horse: 525
  sheep: 364
  spider: 965
  squirrel: 373


Loads and preprocesses a dataset of animal images, initialises ResNet-18 for classification, configures training parameters including an optimiser and error criterion, and then trains the model using PyTorch and mixed precision (if CUDA is available), monitoring loss and accuracy at each step.

In [ ]:
def get_data_loaders(dataset_dir, batch_size=32, num_workers=0):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {
        phase: datasets.ImageFolder(
            os.path.join(dataset_dir, phase),
            transform=data_transforms[phase]
        )
        for phase in ['train', 'test']
    }

    dataloaders = {
        phase: DataLoader(
            image_datasets[phase],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True
        )
        for phase in ['train', 'test']
    }

    dataset_sizes = {phase: len(image_datasets[phase]) for phase in ['train', 'test']}
    class_names = image_datasets['train'].classes

    return dataloaders, dataset_sizes, class_names

def build_model(num_classes):
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scaler, device, num_epochs=5):
    for epoch in range(num_epochs):
        print(f"\n Epoch {epoch+1}/{num_epochs} | LR: {optimizer.param_groups[0]['lr']:.6f}")
        print("-" * 50)

        for phase in ['train', 'test']:
            is_train = (phase == 'train')
            model.train() if is_train else model.eval()

            running_loss = 0.0
            running_corrects = 0

            progress_bar = tqdm(enumerate(dataloaders[phase]), desc=f" {phase.upper()}", leave=False)
            for batch_idx, (inputs, labels) in progress_bar:
                start_time = time.time()
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                optimizer.zero_grad()

                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                if is_train:
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                elapsed = time.time() - start_time
                progress_bar.set_postfix({
                    "loss": f"{loss.item():.4f}",
                    "acc": f"{(preds == labels).float().mean().item():.4f}",
                    "batch_time": f"{elapsed:.4f}s"
                })
                progress_bar.write(f"Processed {batch_idx} in {elapsed:.4f} seconds")

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f" {phase.upper()} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.4f}")

    print("\n The training is complete.")
    return model


dataset_dir = "/content/drive/MyDrive/Kaggle_Datasets/animals10/dataset"
batch_size = 512
num_workers = 10
num_epochs = 15
learning_rate = 0.001
momentum = 0.9
weight_decay = 1e-4


dataloaders, dataset_sizes, class_names = get_data_loaders(dataset_dir, batch_size, num_workers)
print("Classes", class_names)
print("Dataset size", dataset_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_classes = len(class_names)
model = build_model(num_classes)
model = model.to(device)


optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
scaler = torch.amp.GradScaler(device='cuda') if device.type == "cuda" else None


train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scaler, device, num_epochs)

Classes ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']
Dataset size {'train': 21046, 'test': 5241}

 Epoch 1/15 | LR: 0.001000
--------------------------------------------------


Processed 0 in 2.8875 seconds
Processed 1 in 0.1969 seconds


 TRAIN: 3it [01:22, 18.60s/it, loss=2.3016, acc=0.1836, batch_time=0.1541s]

Processed 2 in 0.1541 seconds


 TRAIN: 5it [01:26,  8.19s/it, loss=2.1777, acc=0.2344, batch_time=0.1532s]

...

Processed 1 in 0.0626 seconds
Processed 2 in 0.0625 seconds
Processed 3 in 0.0625 seconds
Processed 4 in 0.0581 seconds


 TEST: 8it [00:09,  1.96it/s, loss=0.1317, acc=0.9629, batch_time=0.0573s]

Processed 5 in 0.0573 seconds
Processed 6 in 0.0572 seconds
Processed 7 in 0.0573 seconds
Processed 8 in 0.0573 seconds


Processed 9 in 0.0573 seconds
Processed 10 in 0.0143 seconds
 TEST | Loss: 0.1217 | Accuracy: 0.9626

 The training is complete.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Saving models

In [ ]:
model_path = "./vision/trained_model_vision"
os.makedirs(model_path, exist_ok=True)
weights_path = os.path.join(model_path, "resnet18_weights.pth")
checkpoint_path = os.path.join(model_path, "resnet18_checkpoint.pth")
torch.save(model.state_dict(), weights_path)
checkpoint = {
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "num_epochs": num_epochs,
}
torch.save(checkpoint, checkpoint_path)